In [1]:
# Imports
import sys
import os
sys.path.append('../..')

# Data Manipulation
import pandas as pd
import openpyxl

In [2]:
# Global Variables
DATA_PATH = "../../data/"

## 1. Arquivos de Layout

In [3]:
# Carrega o arquivo de labels como um arquivo Excel
layout_2010 = pd.ExcelFile('2010_Layout_microdados_Amostra_.xlsx')

In [4]:
# Funções Auxiliares

def remove_unnamed_columns(df) -> pd.DataFrame:
    """
    Remove colunas com nome Unnamed. Estas colunas vem de colunas que foram mescladas no .xlsx original.

    Args:
        df (pd.DataFrame): DataFrame com colunas Unnamed

    Returns:
        pd.DataFrame: DataFrame sem colunas Unnamed
    
    """
    return df.loc[:, ~df.columns.str.contains('^Unnamed')]


def get_values(split: list):
    values = {}
    for val in range(1, len(split)):
        aux = split[val].split('-')
        if len(aux) == 1:
            aux.append(aux[0])
            aux[0] = ' '    
        # aux[1] = aux[1].replace(' ', '')
        values[aux[0]] = aux[1]
    # print(values)
    return values


def split_col_names_values(col: pd.Series): 
    df = pd.DataFrame(columns = ['VAR', 'NOME', 'CHAVE', 'VALOR'])
    for index, value in col.items():
        df_aux = pd.DataFrame(columns = df.columns)
        split = value.split('\n')
        # df_aux['NOME'] = pd.concat([df_aux['NOME'], pd.Series(split[0])])
        if len(split) > 1:
            # series_values = pd.concat([series_values, pd.Series(get_values(split))])
            # df_line = pd.DataFrame(columns = ['NOME', 'CHAVES', 'VALORES'])
            # # df_line['NOME'] = pd.Series(split[0])
            # df_line = pd.Series(get_values)
            # dct = pd.Series(get_values)
            lst = [{'CHAVE': d_key, 'VALOR': d_value, 'VAR': index} for d_key, d_value in get_values(split).items()]
            df_aux = pd.DataFrame(lst)
            df_aux['NOME'] = split[0]
            
        else:
            df_aux['NOME'] = pd.Series(split[0])
            
        df = pd.concat([df, df_aux])
    return df


# def prepare_df(df: pd.DataFrame):
#     df_main = pd.DataFrame(columns = df.columns)
#     df = remove_unnamed_columns(df) 
#     for index, row in df.iterrows():
#         df_row = pd.DataFrame(columns = df.columns)
#         # for col in df.columns.drop('NOME'):
#         #     # print(entry)
#         #     df_row[col] = df[col]
#         df_row = pd.concat([df_row, split_col_names_values(row['NOME'])])
#         # print(df_entry)
#         df_aux = pd.concat([df_aux, df_row])
#     return df, df_aux


def prepare_df(df: pd.DataFrame)-> tuple:
    df = remove_unnamed_columns(df)
    df_vars = split_col_names_values(df.set_index('VAR')['NOME'])
    return df, df_vars

In [5]:
# Carrega as labels referentes aos microdados de domicílios
df_domi = pd.read_excel(layout_2010, sheet_name='DOMI')
df_domi, df_domi_vars = prepare_df(df_domi)
df_domi.head()

,VAR,NOME,POSIÇÃO INICIAL,POSIÇÃO FINAL,INT,DEC,TIPO
0,V0001,UNIDADE DA FEDERAÇÃO:\n11- Rondônia\n12- Acre\...,1,2,2,NaN,A
1,V0002,CÓDIGO DO MUNICÍPIO,3,7,5,NaN,A
2,V0011,ÁREA DE PONDERAÇÃO,8,20,13,NaN,A
3,V0300,CONTROLE,21,28,8,NaN,N
4,V0010,PESO AMOSTRAL,29,44,3,13.0,N


In [6]:
df_domi_vars.head()

,VAR,NOME,CHAVE,VALOR
0,V0001,UNIDADE DA FEDERAÇÃO:,11,Rondônia
1,V0001,UNIDADE DA FEDERAÇÃO:,12,Acre
2,V0001,UNIDADE DA FEDERAÇÃO:,13,Amazonas
3,V0001,UNIDADE DA FEDERAÇÃO:,14,Roraima
4,V0001,UNIDADE DA FEDERAÇÃO:,15,Pará


In [7]:
# # Dataframe de pessoas
# df_pess = pd.read_excel(layout_2010, sheet_name='PESS')
# # Dataframe de emigração  
# df_emig = pd.read_excel(layout_2010, sheet_name='EMIG')
# # Dataframe de mortalidade
# df_mort = pd.read_excel(layout_2010, sheet_name='MORT')

In [8]:
MICRODATA_PATH = DATA_PATH + "microdados/"

dfs = {
    'amostra_domicilios_2010': df_domi,
    # 'amostra_pessoas_2010': df_pess,
    # 'amostra_emigracao_2010': df_emig,
    # 'amostra_mortalidade_2010': df_mort
}

dfs_vars = {
    'amostra_domicilios_2010': df_domi_vars,
    # 'amostra_pessoas_2010': df_pess_vars,
    # 'amostra_emigracao_2010': df_emig_vars,
    # 'amostra_mortalidade_2010': df_mort_vars
}

In [17]:
# Funções auxiliares

def split_by_index(
        input_string: str,
        indexes: list[int]
    ) -> list[str]:
    """
    Splits a string into a list of strings using the indexes provided.

    Args:
        input_string (str): String to be split
        indexes (list[int]): List of indexes indicating where to split the string

    Returns:
        list[str]: List of splitted strings
    """
    return [input_string[i:j] for i, j in zip(indexes, indexes[1:] + [None])]


def extract_line_values(
        file: str, df: pd.DataFrame
    ) -> dict[int, list[str]]:
    """
    Extracts the values from the line of a microdata microdata file.

    Args:
        file (str): Name of the file to be processed
        df (pd.DataFrame): DataFrame containing the indexes of the columns

    Returns:
        dict[int, list[str]]: Dictionary containing the values of each line.
            The keys are the line numbers and the values are lists of strings containing the values not translated.
    """
    # open file
    with open(MICRODATA_PATH + file, 'r') as f:
        # create a counter for indexing the lines
        count = 0
        # create a dictionary to store the values
        values = []
        # read all lines
        lines = f.readlines()
        # iterate over the lines
        for line in lines:
            # split the lines using the indexes from the dataframe
            line_values = split_by_index(line, df['POSIÇÃO INICIAL'].apply(lambda x: int(x) - 1).tolist())
            # save the values in the dictionary
            line_values[-1] = line_values[-1].replace('\n', '')
            values.append(line_values)
            count += 1
    # close the file
        f.close()
    return values


def process_microdata_files(
        file_list: list[str],
        df: dict[str, pd.DataFrame],
        df_vars: dict[str, pd.DataFrame]
    ):
    """
    WORK IN PROGRESS. This function will process all the microdata files
    """
    for file in file_list:
        print(f"Iniciando a extração do arquivo [{file}]")
        values = extract_line_values(file, df)
        print(values)

    # for file in os.listdir(dir_path):
    #     if "amostra" not in file:
    #         continue
    #     # get the name and year of the research from the file name
    #     # the format is amostra_domicilios_YYYY_UF.txt
    #     research = '_'.join(file.split('_')[:3])
    #     try:
    #         # extract the lines from the file
    #         values = extract_line_values(file, dfs[research])
    #         # translate the values
    #         translate_microdata(values, dfs[research])
    #     except KeyError:
    #         # print(f"O arquivo {file} não foi processado.");
    #         continue


# def translate_line_microdata(line_values: str, df: pd.DataFrame, df_vars: pd.DataFrame):
#     df_empty = pd.DataFrame(columns = df_vars['NOME'].unique().tolist())
#     line_dict = {}
#     # for each line, translate the int values to they true values
#     for i in range(len(line_values)):

#         # get the value that will be translaed
#         value = line_values[i]
#         # get the line from the dataframe
#         df_line = df.iloc[i]
#         # get the name of the column (the ith unique column name)
#         df_line_col_name = df_vars['NOME'].unique().tolist()[i]
#         line_dict['VAR'] = df_line['VAR']
#         # verify if the value is "translatable"
#         if df_line['TIPO'] == 'C' or df_line['VAR'] == 'V0001':
#             # if it is, get the possible values
#             possible_values = df_vars[df_vars['VAR'] == df_line['VAR']]
#             # get the value from the possible values
#             translated_value = possible_values[possible_values['CHAVE'] == value]
#             try:
#                 line_dict[df_line_col_name] = str(translated_value['VALOR'][0])
#                 # print(str(translated_value['VALOR']))
#             except KeyError:
#                 print(df_line)
#                 print(str(translated_value['VALOR'].iloc[0]))
#                 line_dict[df_line_col_name] = str(translated_value['VALOR'].iloc[0])
#         else:
#             line_dict[df_line_col_name] = value
#         df_line = pd.concat([df_empty, pd.DataFrame(line_dict, index=[0])])
#     return df_line


def translate_line_microdata(line_values: str, df: pd.DataFrame, df_vars: pd.DataFrame):
    # iterate over all the columns of the microdata line
    for i in range(len(line_values)):
        # get the value that will be translaed
        value = line_values[i]
        # get the var code
        df_line = dict(df.iloc[i])
        # verify if the value is "translatable"
        if (df_line['TIPO'] == 'C' or df_line['VAR'] == 'V0001') and df_line['VAR'][0] != 'M':
            # if it is, get the possible values
            possible_values = df_vars[df_vars['VAR'] == df_line['VAR']]
            try:
                translated_value = possible_values[possible_values['CHAVE'] == value]['VALOR'].values[0]
            except IndexError:
                translated_value = value            

a = translate_line_microdata(sc_line_values[0], dfs['amostra_domicilios_2010'], dfs_vars['amostra_domicilios_2010'])
a

0     11
1     12
2     13
3     14
4     15
5     16
6     17
7     21
8     22
9     23
10    24
11    25
12    26
13    27
14    28
15    29
16    31
17    32
18    33
19    35
20    41
21    42
22    43
23    50
24    51
25    52
26    53
Name: CHAVE, dtype: object
0    1
1    2
2    3
3    4
4    5
Name: CHAVE, dtype: object
0    1
1    2
2     
Name: CHAVE, dtype: object
0     01
1     02
2     03
3     04
4     05
5     06
6     07
7     08
8     09
9     10
10      
Name: CHAVE, dtype: object
0    1
1    2
2    3
3     
Name: CHAVE, dtype: object
0    1
1    2
2    3
3     
Name: CHAVE, dtype: object
0    1
1    2
2     
Name: CHAVE, dtype: object
0    1
1    2
2     
Name: CHAVE, dtype: object
0    1
1    2
2     
Name: CHAVE, dtype: object
0    1
1    2
2     
Name: CHAVE, dtype: object
0    1
1    2
2     
Name: CHAVE, dtype: object
0    1
1    2
2     
Name: CHAVE, dtype: object
0    1
1    2
2     
Name: CHAVE, dtype: object
0    1
1    2
2     
Name: CHAVE, dtype: object


In [10]:
%%time
file = "amostra_domicilios_2010_SC.txt"
sc_line_values = extract_line_values(file, dfs['amostra_domicilios_2010'])

CPU times: total: 29 s
Wall time: 51 s


In [ ]:
df_domi_vars

,VAR,NOME,CHAVE,VALOR
0,V0001,UNIDADE DA FEDERAÇÃO:,11,Rondônia
1,V0001,UNIDADE DA FEDERAÇÃO:,12,Acre
2,V0001,UNIDADE DA FEDERAÇÃO:,13,Amazonas
3,V0001,UNIDADE DA FEDERAÇÃO:,14,Roraima
4,V0001,UNIDADE DA FEDERAÇÃO:,15,Pará
...,...,...,...,...
6,V1005,Situação do setor,7,Aglomerado rural (outros)
7,V1005,Situação do setor,8,Área rural exclusive aglomerado rural
8,V1005,Situação do setor,,
9,V1005,Situação do setor,1,Masculino


In [ ]:
df_domi

,VAR,NOME,POSIÇÃO INICIAL,POSIÇÃO FINAL,INT,DEC,TIPO
0,V0001,UNIDADE DA FEDERAÇÃO:\n11- Rondônia\n12- Acre\...,1,2,2,NaN,A
1,V0002,CÓDIGO DO MUNICÍPIO,3,7,5,NaN,A
2,V0011,ÁREA DE PONDERAÇÃO,8,20,13,NaN,A
3,V0300,CONTROLE,21,28,8,NaN,N
4,V0010,PESO AMOSTRAL,29,44,3,13.0,N
...,...,...,...,...,...,...,...
71,M0301,MARCA DE IMPUTAÇÃO NA V0301: \n1- Sim\n2- Não,168,168,1,NaN,C
72,M0401,MARCA DE IMPUTAÇÃO NA V0401: \n1- Sim\n2- Não,169,169,1,NaN,C
73,M0402,MARCA DE IMPUTAÇÃO NA V0402: \n1- Sim\n2- Não,170,170,1,NaN,C
74,M0701,MARCA DE IMPUTAÇÃO NA V0701: \n1- Sim\n2- Não,171,171,1,NaN,C


In [ ]:
# with open(MICRODATA_PATH + file, 'r') as f:
#     lines = f.readlines()
#     print(lines[0], lines[1], lines[2], lines[3], lines[4], sep='')